<a href="https://colab.research.google.com/github/mirachakshu/nlp_begin/blob/master/nnlm_tensor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
# code by Tae Hwan Jung @graykode
import tensorflow as tf
import numpy as np


In [0]:
tf.reset_default_graph()

sentences = [ "i like dog", "i love coffee", "i hate milk"]

In [6]:
word_list = " ".join(sentences).split()
print (word_list)

['i', 'like', 'dog', 'i', 'love', 'coffee', 'i', 'hate', 'milk']


In [9]:
word_list = list(set(word_list))
print (word_list)

['love', 'coffee', 'dog', 'milk', 'i', 'hate', 'like']


In [10]:
word_dict = {w: i for i, w in enumerate(word_list)}
print (word_dict)
number_dict = {i: w for i, w in enumerate(word_list)}
print (number_dict)

{'love': 0, 'coffee': 1, 'dog': 2, 'milk': 3, 'i': 4, 'hate': 5, 'like': 6}
{0: 'love', 1: 'coffee', 2: 'dog', 3: 'milk', 4: 'i', 5: 'hate', 6: 'like'}


In [2]:
n_class = len(word_dict) # number of Vocabulary

# NNLM Parameter
n_step = 2 # number of steps ['i like', 'i love', 'i hate']
n_hidden = 2 # number of hidden units

def make_batch(sentences):
    input_batch = []
    target_batch = []

    for sen in sentences:
        word = sen.split()
        input = [word_dict[n] for n in word[:-1]]
        target = word_dict[word[-1]]

        input_batch.append(np.eye(n_class)[input])
        target_batch.append(np.eye(n_class)[target])

    return input_batch, target_batch

# Model
X = tf.placeholder(tf.float32, [None, n_step, n_class]) # [batch_size, number of steps, number of Vocabulary]
Y = tf.placeholder(tf.float32, [None, n_class])

input = tf.reshape(X, shape=[-1, n_step * n_class]) # [batch_size, n_step * n_class]
H = tf.Variable(tf.random_normal([n_step * n_class, n_hidden]))
d = tf.Variable(tf.random_normal([n_hidden]))
U = tf.Variable(tf.random_normal([n_hidden, n_class]))
b = tf.Variable(tf.random_normal([n_class]))

tanh = tf.nn.tanh(d + tf.matmul(input, H)) # [batch_size, n_hidden]
model = tf.matmul(tanh, U) + b # [batch_size, n_class]

cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=model, labels=Y))
optimizer = tf.train.AdamOptimizer(0.001).minimize(cost)
prediction =tf.argmax(model, 1)

# Training
init = tf.global_variables_initializer()
sess = tf.Session()
sess.run(init)

input_batch, target_batch = make_batch(sentences)

for epoch in range(5000):
    _, loss = sess.run([optimizer, cost], feed_dict={X: input_batch, Y: target_batch})
    if (epoch + 1)%1000 == 0:
        print('Epoch:', '%04d' % (epoch + 1), 'cost =', '{:.6f}'.format(loss))

# Predict
predict =  sess.run([prediction], feed_dict={X: input_batch})

# Test
input = [sen.split()[:2] for sen in sentences]
print([sen.split()[:2] for sen in sentences], '->', [number_dict[n] for n in predict[0]])

['i', 'like', 'dog', 'i', 'love', 'coffee', 'i', 'hate', 'milk']
Epoch: 1000 cost = 0.565055
Epoch: 2000 cost = 0.316271
Epoch: 3000 cost = 0.217067
Epoch: 4000 cost = 0.154149
Epoch: 5000 cost = 0.025315
[['i', 'like'], ['i', 'love'], ['i', 'hate']] -> ['dog', 'coffee', 'milk']
